<a href="https://colab.research.google.com/github/qahathaway/Automated_Segmentation_PTM-IMAT-SAT/blob/main/Pectoralis_Semantic_Segmentation_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py37" --user

In [5]:
!python --version

Python 3.7.6


In [3]:
!git clone https://github.com/qahathaway/Automated_Segmentation_PTM-IMAT-SAT.git

Cloning into 'Automated_Segmentation_PTM-IMAT-SAT'...
remote: Enumerating objects: 1991, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1991 (delta 7), reused 2 (delta 0), pack-reused 1978
Receiving objects: 100% (1991/1991), 190.28 MiB | 21.22 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Filtering content: 100% (2/2), 416.60 MiB | 39.30 MiB/s, done.


In [4]:
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Selected
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Output
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer

# Step 3: Select the frame directly above the aortic arch

In [5]:
### Load Essential Packages ###
import numpy as np
import pandas as pd
import os
import cv2
from imutils import paths
import argparse
import glob
from glob import iglob

n_white_pix = []
names = []
file_count = []
index = 0

### Select Frame Directly Above the Aorta ###
for filename in sorted(glob.glob('/content/Automated_Segmentation_PTM-IMAT-SAT/Example/**/axial_*.png'), key=lambda name: (name[53:61], int(os.path.basename(name)[14:-4]))):
    image = cv2.imread(filename)
    names.append(filename)
    pathlen, filenamed = os.path.split(filename)
    file_count = len(glob.glob1(pathlen,"axial_*.png"))
    image[image <= 80] = 0
    image[image >= 140] = 0
    cropped = image[165:360, 180:332]
    n_white_pix = np.append(n_white_pix, np.sum(cropped > 0))
    index += 1

### Select the Array of Images to Search and Apply Pixel Intensities to Stratify ###
    if index == file_count:
        lower = int(file_count*0.6)
        upper = int(file_count*0.85)
        minimum = (np.argmin(n_white_pix[lower:upper]) + lower)
        midline = cv2.imread(names[minimum])
        path, filenames = os.path.split(names[minimum])
        path2 = os.path.basename(path)
        cv2.imwrite('/content/Automated_Segmentation_PTM-IMAT-SAT/Selected/' + path2 + "_" + filenames, midline)

        n_white_pix = []
        names = []
        file_count = []
        index = 0

    else:
        continue


# Step 4: Custom semantic segmentation using Pixellib and Mask R-CNN for all remaining participants

In [10]:
!pip install tensorflow==2.5.3
!pip install keras==2.4.3
!pip install numpy==1.19.5
!pip install pillow==8.2.0
!pip install PyQt5==5.15.5
!pip install pixellib==0.7.1
!pip install scikit-image==0.18.0

     |████████████████████████████████| 460.3 MB 5.1 kB/s 
     |████████████████████████████████| 4.0 MB 21.9 MB/s 
     |████████████████████████████████| 304 kB 48.5 MB/s 
     |████████████████████████████████| 14.8 MB 29.1 MB/s 
     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 462 kB 30.6 MB/s 
     |████████████████████████████████| 1.2 MB 43.8 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 6.0 MB 34.8 MB/s 
     |████████████████████████████████| 132 kB 43.9 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 4.0 MB 18.3 MB/s 
     |████████████████████████████████| 4.9 MB 26.7 MB/s 
     |████████████████████████████████| 781 kB 34.3 MB/s 
     |████████████████████████████████| 233 kB 35.8 MB/s 
     |████████████████████████████████| 93 kB 2.1 MB/s 
     |████████████████

     |████████████████████████████████| 3.0 MB 4.2 MB/s 
ERROR: sahi 0.11.14 has requirement tqdm>=4.48.2, but you'll have tqdm 4.42.1 which is incompatible.
ERROR: imageio 2.31.1 has requirement pillow>=8.3.2, but you'll have pillow 8.2.0 which is incompatible.
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
     |████████████████████████████████| 29.2 MB 34.6 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.19.3
    Uninstalling scikit-image-0.19.3:
      Successfully uninstalled scikit-image-0.19.3


In [2]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [7]:
print("User Current Version:-", sys.version)

User Current Version:- 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]


In [9]:
!pip install tensorboard==2.5.0

     |████████████████████████████████| 6.0 MB 5.0 MB/s 
     |████████████████████████████████| 152 kB 57.6 MB/s 
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.1
    Uninstalling cachetools-5.3.1:
      Successfully uninstalled cachetools-5.3.1
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.21.0
    Uninstalling google-auth-2.21.0:
      Successfully uninstalled google-auth-2.21.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.2
    Uninstalling tensorboard-2.11.2:
      Successfully uninstalled tensorboard-2.11.2


In [8]:
!pip freeze

absl-py==0.15.0
aiohttp==3.8.1
aiosignal==1.3.1
antlr4-python3-runtime==4.9.3
anyio==3.5.0
argon2-cffi==20.1.0
asn1crypto==1.3.0
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
attrs==22.1.0
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.11.1
black==23.3.0
bleach==4.1.0
cached-property==1.5.2
cachetools==5.3.1
certifi==2023.5.7
cffi==1.14.0
chardet==3.0.4
charset-normalizer==2.0.4
click==8.1.4
cloudpickle==2.2.1
conda==23.1.0
conda-package-handling==1.6.0
cryptography==3.4.8
cycler==0.11.0
Cython==0.29.36
dataclasses==0.6
debugpy==1.5.1
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.4
fastjsonschema==2.16.2
fire==0.5.0
flatbuffers==1.12
fonttools==4.38.0
frozenlist==1.3.3
future==0.18.3
fvcore==0.1.5.post20221221
gast==0.4.0
google-auth==2.21.0
google-auth-oauthlib==0.4.6
google-colab==1.0.0
google-pasta==0.2.0
grpcio==1.34.1
h5py==3.1.0
hydra-core==1.3.2
idna==2.8
imageio==2.31.1
imantics==0.1.12
imgaug==0.4.0
importlib-metadata==4.11.3
importlib-resources==5.2.0
iopath==

In [3]:
### Load Essential Packages ###
import pixellib
from pixellib.instance import custom_segmentation

### Custom Segmentation of Left PMAT, Right PMAT, and SAT ###
segment_image = custom_segmentation()
segment_image.inferConfig(num_classes = 3, class_names= ["BG", "PMR", "PML", "SAT"], detection_threshold = 0.9, network_backbone = "resnet50")
segment_image.load_model("/content/Automated_Segmentation_PTM-IMAT-SAT/COCO/95_MSK_3_Class.h5")

Instructions for updating:
Use fn_output_signature instead


In [4]:
### Load Essential Packages ###
import numpy as np
from PIL import Image
import cv2
import os

### Set Directories ###
inputdir = '/content/Automated_Segmentation_PTM-IMAT-SAT/Selected/'
outdir = '/content/Automated_Segmentation_PTM-IMAT-SAT/Output/'
outdir1 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PMR_Mask_'
outdir2 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PMR_'
outdir3 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PML_Mask_'
outdir4 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PML_'
outdir5 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/SAT_Mask_'
outdir6 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/SAT_'

test_list = [ f for f in  os.listdir(inputdir)]

### Inference of Left PMAT, Right PMAT, and SAT Images Based on Mask R-CNN Model ###
for f in sorted (test_list):
    if f.endswith('.png'):
        segmask, output = segment_image.segmentImage(inputdir + f, extract_segmented_objects = False, mask_points_values = True, show_bboxes=False)
        class_list = segmask['class_ids'].tolist()
        class_str = ''.join(str(x) for x in class_list)
        try:
            if class_str == '321':
                c,b,a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '312':
                c,a,b = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '231':
                b,c,a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '213':
                b,a,c = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '132':
                a,c,b = [ [individualArray] for individualArray in segmask['masks'] ]
            else:
                a,b,c = [ [individualArray] for individualArray in segmask['masks'] ]
        except:
            pass

        cv2.imwrite(outdir + f, output)
        a1 = np.array(a)
        b1 = np.array(b)
        c1 = np.array(c)
        for i in a1.tolist():
            mask1 = np.asarray(Image.open(inputdir + f))
            mask11 =  mask1[:,:,:3]
            img1 = cv2.fillPoly(mask1, np.array(i), color=(0, 0, 0))
            maskB = Image.fromarray(mask11)
        for i in a1.tolist():
            mask2 = np.asarray(Image.open(inputdir + f))
            mask22 =  mask2[:,:,:3]
            img2 = cv2.fillPoly(mask2, np.array(i), color=(255, 255, 255))
            maskW = Image.fromarray(mask22)
            ROI_PMR_Mask = np.subtract(maskW, maskB)
            ROI_PMR = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB)
            cv2.imwrite(outdir1 + f, ROI_PMR_Mask)
            cv2.imwrite(outdir2 + f, ROI_PMR)
        for i in b1.tolist():
            mask3 = np.asarray(Image.open(inputdir + f))
            mask33 =  mask3[:,:,:3]
            img3 = cv2.fillPoly(mask3, np.array(i), color=(0, 0, 0))
            maskB2 = Image.fromarray(mask33)
        for i in b1.tolist():
            mask4 = np.asarray(Image.open(inputdir + f))
            mask44 =  mask4[:,:,:3]
            img4 = cv2.fillPoly(mask4, np.array(i), color=(255, 255, 255))
            maskW2 = Image.fromarray(mask44)
            ROI_SAT_Mask = np.subtract(maskW2, maskB2)
            ROI_SAT = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB2)
            cv2.imwrite(outdir3 + f, ROI_SAT_Mask)
            cv2.imwrite(outdir4 + f, ROI_SAT)
        for i in c1.tolist():
            mask5 = np.asarray(Image.open(inputdir + f))
            mask55 =  mask5[:,:,:3]
            img5 = cv2.fillPoly(mask5, np.array(i), color=(0, 0, 0))
            maskB3 = Image.fromarray(mask55)
        for i in c1.tolist():
            mask6 = np.asarray(Image.open(inputdir + f))
            mask66 =  mask6[:,:,:3]
            img6 = cv2.fillPoly(mask6, np.array(i), color=(255, 255, 255))
            maskW3 = Image.fromarray(mask66)
            ROI_PML_Mask = np.subtract(maskW3, maskB3)
            ROI_PML = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB3)
            cv2.imwrite(outdir5 + f, ROI_PML_Mask)
            cv2.imwrite(outdir6 + f, ROI_PML)

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


NameError: ignored

Step 6: Determine the total number of pixels within the ROI

In [ ]:
### Load Essential Packages ###
import cv2
import os
import numpy as np
import glob
from glob import iglob
import csv

### Count White Pixels in Image ###
for filename in sorted(glob.glob('path/to/ROI/**/*.png')):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    n_white_pix = np.sum(img == 255)
    average = img[img!=0].mean()
    path, filenames = os.path.split(filename)
    path2 = os.path.basename(path)
    if "Mask" in filenames:
        name_type = "Mask"
    else:
        name_type = "Image"
    if "PML_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Left"
    elif "PMR_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Right"
    elif "PML-PATL_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Left Subtracted from Intermuscular Adipose Tissue, Left"
    elif "PMR-PATR_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Right Subtracted from Intermuscular Adipose Tissue, Right"
    elif "PML-PATL_MESA" in filenames:
        name_region = "Pectoralis Muscle, Left Subtracted from Intermuscular Adipose Tissue, Left"
    elif "PMR-PATR_MESA" in filenames:
        name_region = "Pectoralis Muscle, Right Subtracted from Intermuscular Adipose Tissue, Right"
    elif "EMLL_Mask_" in filenames:
        name_region = "Extramyocellular Lipid, Left"
    elif "EMLR_Mask_" in filenames:
        name_region = "Extramyocellular Lipid, Right"
    elif "SAT_Mask_" in filenames:
        name_region = "Subcutaneous Adipose Tissue"
    elif "PATL_Mask_" in filenames:
        name_region = "Intermuscular Adipose Tissue, Left"
    elif "PATR_Mask_" in filenames:
        name_region = "Intermuscular Adipose Tissue, Right"
    fields=[path2, name_type, name_region, filenames, n_white_pix, average]
    with open('path/to/Measurements_Mask.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)